In [2]:
from magpie import grasp as gt
import rtde_control
import rtde_receive
from magpie.motor_code import Motors
from magpie import ur5 as ur5
import time
import numpy as np
import copy
import magpie
from magpie.gripper import Gripper

In [2]:
import importlib
importlib.reload(gt)
importlib.reload(ur5)
importlib.reload(magpie.gripper)

<module 'magpie.gripper' from '/home/will/MAGPIE/magpie/gripper.py'>

In [15]:
# tmat = np.array([[ 0.91574736, -0.35602909,  0.18614527, -0.02848649],
#        [-0.40073104, -0.77640133,  0.48643151,  0.14332736],
#        [ 0.02866033,  0.52004256,  0.85365937,  0.33750796],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

tmat = np.array([[-0.95229089,  0.08163851,  0.29407007, -0.0540869 ],
       [-0.17853089, -0.93050792, -0.31981514, -0.06974001],
       [ 0.2475253 , -0.35705764,  0.90068922,  0.38554392],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

aperture = 37
# z_offset = G.aperture_to_z(aperture)/1000.0

mmc = copy.deepcopy(tmat[:3, 3])
# grasp_pose = [mmc[1], -mmc[0], mmc[2]]
grasp_pose = mmc
tmat[:3, 3] = [0, 0, 0]
homePose = None
z_offset = 105/1000.0
z_offset -= -0.02
z_offset

0.125

In [18]:
# orient gripper in place OR move gripper to cartesian pos
sleepRate = 1.5
ur = ur5.UR5_Interface()
try:
    ur.start()
    # homePose = ur.get_tcp_pose()
    homePose = ur.getPose()
    currentPose = ur.getPose()
    desiredPose = np.matmul(np.array(currentPose), tmat)
    print("Desired Pose: ", desiredPose)
    # in-place (or best attempt) re-orientation
    # ur.moveL(desiredPose)
    ## move to cartesian pos
    gt.move_to(grasp_pose, ur, z_offset=z_offset)    
    print("Done moving to block")
    time.sleep(sleepRate)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Desired Pose:  [[-0.267  0.915 -0.304 -0.27 ]
 [-0.955 -0.292 -0.041 -0.082]
 [ 0.127 -0.279 -0.952  0.233]
 [ 0.     0.     0.     1.   ]]
[[ 0.081 -0.935 -0.346 -0.27 ]
 [-0.997 -0.072 -0.039 -0.082]
 [ 0.011  0.348 -0.937  0.233]
 [ 0.     0.     0.     1.   ]]
P_goal:
[-0.058  0.071 -0.268]
Done moving to block


In [19]:
# move home
ur = ur5.UR5_Interface()
try:
    ur.start()
    # time.sleep(sleepRate)
    ur.moveL(homePose)
    time.sleep(sleepRate * 3)
    ur.stop()
except Exception as e:
    ur.stop()

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
[TxRxResult] There is no status packet!
[TxRxResult] There is no status packet!


In [409]:
# orient and move gripper to cartesian pos
sleepRate = 1.5
ur = ur5.UR5_Interface()
try:
    servoPort = "/dev/ttyACM0"
    ur.start()
    time.sleep(sleepRate)
    homePose = ur.get_tcp_pose()    
    '''
    stopping point 2/15:
    this works so far
    take home pos
    add translation on the gripper frame to the goal x, y
    apply tmat to the goal
    move to goal (translates on x, y and orients at the same time)
    ## does this address the fact that orientation is not achievable in standstill?

    then: take orig Z (from original gripper frame): [0, 0, dZ]
    transform by tmat (the current orientation of the gripper)
    add transformed dZ to the goal position
    move
    works!
    '''
    dX,dY,dZ = gt.get_world_frame(grasp_pose, ur, z_offset=z_offset)

    goal1 = copy.deepcopy(homePose)
    goal1 = np.array(goal1)
    # goal1[:3, 3] += [dX, dY, 0]
    goal1[:3, 3] += [dX, dY, dZ]
    goal1 = np.array(goal1) @ tmat
    ur.moveL(goal1)
    time.sleep(sleepRate)

    # todo: take currentPose and validate orientation?
    goal2 = goal1
    posd = [0, 0, dZ] @ tmat[:3, :3]
    goal2 = np.array(goal2)
    goal2[:3, 3] += posd
    # ur.moveL(goal2)
    time.sleep(sleepRate)
    '''
    end of working section
    '''
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
[[ 0.957 -0.222 -0.186 -0.056]
 [-0.274 -0.903 -0.332 -0.496]
 [-0.094  0.368 -0.925  0.181]
 [ 0.     0.     0.     1.   ]]
P_goal:
[-0.061 -0.04  -0.246]


RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
